In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os

# Solicitar el archivo txt al usuario
from google.colab import files

print("Por favor, sube el archivo .txt con las intervenciones del parlamentario.")
uploaded = files.upload()

# Leer el archivo .txt
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = text.replace('\n', ' ')  # Reemplazar saltos de línea por espacios
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    return text

text = preprocess_text(text)
words = text.split()  # Dividir el texto en palabras únicas
vocab = sorted(set(words))  # Crear vocabulario único

# Crear diccionarios de mapeo
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convertir el texto a índices
encoded_text = [word_to_idx[word] for word in words]

# Preparar los datos para el modelo
sequence_length = 10  # Aumentar la longitud de las secuencias
X = []
y = []

for i in range(len(encoded_text) - sequence_length):
    X.append(encoded_text[i:i+sequence_length])
    y.append(encoded_text[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Construir el modelo GRU con 4 capas, unidades entre 400 y 512 y dropout del 30-40%
vocab_size = len(vocab)
embedding_dim = 128  # Aumentar la dimensión de los embeddings
gru_units = 512  # Usar unidades más grandes para mejorar el aprendizaje

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    GRU(gru_units, return_sequences=True),
    Dropout(0.4),  # Dropout del 40%
    GRU(gru_units, return_sequences=True),
    Dropout(0.4),
    GRU(gru_units, return_sequences=True),
    Dropout(0.4),
    GRU(gru_units),
    Dropout(0.4),
    Dense(vocab_size, activation='softmax')
])

# Compilar el modelo con el optimizador Adam y una tasa de aprendizaje de 0.00005
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), metrics=['accuracy'])

# Añadir callback para guardar el modelo con el mejor rendimiento
checkpoint = ModelCheckpoint('modelo_GRU_parlamentarioCasado.keras', monitor='loss', save_best_only=True)

# Entrenamiento largo, con más épocas y callback
print("Entrenando el modelo durante más épocas...")
model.fit(X, y, epochs=220, batch_size=256, verbose=1, callbacks=[checkpoint])

# Función para generar texto con temperatura
def generate_text(model, start_sequence, max_words=100, temperature=1.0):
    input_sequence = [word_to_idx[word] for word in start_sequence.split() if word in word_to_idx]
    generated = start_sequence

    for _ in range(max_words):
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], maxlen=sequence_length)
        predictions = model.predict(input_padded, verbose=0).flatten()
        predictions = np.log(predictions + 1e-9) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        predicted_idx = np.random.choice(len(probabilities), p=probabilities)
        predicted_word = idx_to_word[predicted_idx]

        generated += ' ' + predicted_word
        input_sequence.append(predicted_idx)
        input_sequence = input_sequence[1:]

        # Finalizar si se alcanza un final lógico
        if predicted_word in ['.', '!', '?']:
            break

    return generated

# Interactuar con el modelo sin preguntar la longitud
print("¡Puedes comenzar a interactuar con el modelo!")
while True:
    prompt = input("Introduce el inicio del texto (o escribe 'salir' para terminar): ")
    if prompt.lower() == 'salir':
        print("¡Hasta luego!")
        break
    print("Respondiendo...")
    print(generate_text(model, prompt, max_words=100, temperature=0.8))


Por favor, sube el archivo .txt con las intervenciones del parlamentario.


Saving intervencionesCasado.txt to intervencionesCasado (12).txt
Entrenando el modelo durante más épocas...
Epoch 1/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.0354 - loss: 8.1033
Epoch 2/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0382 - loss: 6.4498
Epoch 3/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0394 - loss: 6.3101
Epoch 4/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0416 - loss: 6.3006
Epoch 5/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0446 - loss: 6.3126
Epoch 6/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.0436 - loss: 6.2743
Epoch 7/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0427 - loss: 6.2867
Epoch 8/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0430 - loss: 6.2939
Epoch 9/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.0427 - loss: 6.2572
Epoch 10/220
72/72 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.0440 - loss: 6.2776
Epoch 11/220
72/72 ━━━━━━━━━━━━